# Run this notebook on CUDA and with GPUs as it involves Finetuning and Transfer Learning

In [7]:
# Note: CUDA® is a parallel computing platform and programming model developed by NVIDIA for general computing on graphical processing units (GPUs).

## Sparse Transfer

In [2]:
# !pip install sparseml
# !pip install sparseml[transformers]

### create a teacher for the desired text classification dataset, we will fine-tune a dense BERT model from the SparseZoo

## NLP

In [1]:
# models are saved locally

In [3]:
!sparseml.transformers.text_classification \
    --output_dir models/teacher \
    --model_name_or_path "zoo:nlp/masked_language_modeling/bert-base/pytorch/huggingface/wikipedia_bookcorpus/base-none" \
    --recipe "zoo:nlp/masked_language_modeling/bert-base/pytorch/huggingface/wikipedia_bookcorpus/base-none?recipe_type=transfer-text_classification" \
    --recipe_args '{"init_lr":0.00003}' \
    --task_name sst2 \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 --per_device_eval_batch_size 32 \
    --do_train --do_eval --evaluation_strategy epoch --fp16  \
    --save_strategy epoch --save_total_limit 1

2024-01-17 18:28:50 sparseml.transformers.text_classification WARNING  Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: True

Generating test split: 100%|█████| 1821/1821 [00:00<00:00, 723484.66 examples/s]
[WARNING|modeling_utils.py:3777] 2024-01-17 18:28:54,309 >> Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/jupyter/.cache/sparsezoo/neuralmagic/bert-base-wikipedia_bookcorpus-base/training and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-01-17 18:28:54 sparseml.transformers.utils.model INFO     Loaded model from /home/jupyter/.cache/sparsezoo/neuralmagic/bert-base-wikipedia_bookcorpus-base/training with 109483778 total params. Of those there are 85526016 prunable params which have 0.0 avg sparsity.
2024-

### With the teacher model trained, it is ready to be distilled into a sparsified student model.

In [6]:
!sparseml.transformers.text_classification \
    --output_dir models/sparsified \
    --model_name_or_path "zoo:nlp/masked_language_modeling/distilbert-none/pytorch/huggingface/wikipedia_bookcorpus/pruned80_quant-none-vnni" \
    --recipe "zoo:nlp/sentiment_analysis/distilbert-none/pytorch/huggingface/sst2/pruned80_quant-none-vnni" \
    --distill_teacher models/teacher \
    --task_name sst2 \
    --max_seq_length 128 \
    --per_device_train_batch_size 32 --per_device_eval_batch_size 32 \
    --do_train --do_eval --evaluation_strategy epoch --fp16 \
    --save_strategy epoch --save_total_limit 1

2024-01-17 19:49:22 sparseml.transformers.text_classification WARNING  Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: True
[WARNING|modeling_utils.py:3777] 2024-01-17 19:49:25,452 >> Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /home/jupyter/.cache/sparsezoo/neuralmagic/distilbert-wikipedia_bookcorpus-pruned80.4block_quantized/training and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-01-17 19:49:25 sparseml.transformers.utils.model INFO     Loaded student from /home/jupyter/.cache/sparsezoo/neuralmagic/distilbert-wikipedia_bookcorpus-pruned80.4block_quantized/training with 66955010 total params. Of those there are 43058688 prunable params which have 78.90131719758855 avg sparsity.
2024-01-17 19:49:25 sparseml

#### Exporting for inference

In [1]:
!sparseml.transformers.export_onnx \
    --model_path models/sparsified \
    --task 'text-classification' --finetuning_task sst2 \
    --sequence_length 128

2024-01-18 12:41:33 sparseml.transformers.export INFO     Attempting onnx export for model at /home/jupyter/neural-magic/models/sparsified for task text-classification
2024-01-18 12:41:36 sparseml.transformers.utils.model WARNING  QAT state detected, ignore any loading errors, weights will reload after SparseML recipes have been applied /home/jupyter/neural-magic/models/sparsified
Some weights of the model checkpoint at /home/jupyter/neural-magic/models/sparsified were not used when initializing DistilBertForSequenceClassification: ['distilbert.transformer.layer.2.attention.attention_scores_matmul.input_quant_stubs.1.activation_post_process.activation_post_process.eps', 'distilbert.transformer.layer.1.attention.attention_scores_matmul.output_quant_stubs.0.activation_post_process.activation_post_process.eps', 'distilbert.transformer.layer.4.attention.context_layer_matmul.input_quant_stubs.0.activation_post_process.fake_quant_enabled', 'distilbert.transformer.layer.0.attention.attention_